# Workflow for the CRC1333 project B07 - Technical Chemistry

**Import necessary packages.**

In [4]:
import sklearn

import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
import pandas as pd

from datetime import datetime
from sdRDM import DataModel
from modules import GCParser
from modules import GstaticParser
from modules import MFMParser
from modules import Calculator
from pathlib import Path
# from DEXPI2sdRDM import DEXPI2sdRDM

### Set up the data model and the correct path to the working directory.

**Load data model from markdown file.**

In [5]:
model = DataModel.from_markdown('specifications/datamodel_b07_tc.md')

In [6]:
# model = DataModel.from_git(url='https://github.com/FAIRChemistry/datamodel_b07_tc.git')

**Set path to current working directory.**

In [7]:
path_to_dataset = Path.cwd()

**Initialize Dataset of the project.**

In [8]:
b07 = model.Dataset()
b07.__dict__

{'id': 'dataset0', 'general_information': None, 'experiments': []}

**Store general information about the project like the title of the project, the authors and a project description into the data model.**

In [9]:
gen_inf = model.GeneralInformation()
title = 'Electrocatalytic CO2-reduction on carbon'
gen_inf.title = title
description = 'The aim of this project is to blablabla'
gen_inf.description = description
author_1 = model.Author(name= 'Richard Schömig', affiliation = 'University of Stuttgart')
author_2 = model.Author(name= 'Maximilian Schmidt', affiliation = 'University of Stuttgart')
gen_inf.authors = [author_1, author_2]
b07.general_information= gen_inf
# b07.general_information = gen_inf
# gen_inf.add_to_authors(name='test')
# gen_inf.add_to_authors(name='test')
print(b07.__dict__)

{'id': 'dataset0', 'general_information': GeneralInformation(id='generalinformation0', title='Electrocatalytic CO2-reduction on carbon', description='The aim of this project is to blablabla', authors=[Author(id='author0', name='Richard Schömig', affiliation='University of Stuttgart'), Author(id='author1', name='Maximilian Schmidt', affiliation='University of Stuttgart')]), 'experiments': []}


In [10]:
# calculation=model.Calculation(calibrations= list(cali_dict.values()))
# experiment_1=model.Experiment(calculations=calculation)
# b07=model.Dataset(experiments=[experiment_1])
# # b07.add_experiment_to_experiments(experiment_1)
# b07.general_information=general_information
# b07.__dict__
# # b07

**Instantiate an experiment objects which holds all the information about one single experiment.**

In [11]:
experiment = model.Experiment()

### Load metadata into the datamodel.

**Load P&ID file.**

In [12]:
# filename = "column"
# path = "./f'{filename}'"

**Load DEXPI conform P&ID file into the data model using the ``DEXPI2sdRDM`` module.**

In [13]:
# DEXPI2sdRDM(
# )

**Create an instance of the ``GstaticParser`` to parse Gamry output files and show available files in the selected directory.**

In [14]:
gstaticparser = GstaticParser(path_to_dataset / 'data')
dict_of_gstatic_files = gstaticparser.enumerate_available_files()
for index, gstatic_file in dict_of_gstatic_files.items():
    print(f"{index}: {gstatic_file}")
# available_files = gstaticparser.available_files
# print(available_files)

0: GSTATIC(Potentiostat)


In [15]:
# measurements = model.Measurement()

**Chose specific file and extract the metadata from it using the ``GstaticParser``.**

In [16]:
gstatic_file = dict_of_gstatic_files[0]
gstatic_meta_df, gstatic_meta = gstaticparser.extract_metadata(gstatic_file)
# ExPot = model.PotentiostaticMeasurement()
# experiment.add_to_measurements(gstatic_meta)
experiment.measurements = [gstatic_meta]
# gstatic_meta_df
# experiment.__dict__
# b07.add_to_experiments(experiment)
b07.experiments=[experiment]
print(b07.json())

{
  "id": "dataset0",
  "general_information": {
    "id": "generalinformation0",
    "title": "Electrocatalytic CO2-reduction on carbon",
    "description": "The aim of this project is to blablabla",
    "authors": [
      {
        "id": "author0",
        "name": "Richard Sch\u00f6mig",
        "affiliation": "University of Stuttgart"
      },
      {
        "id": "author1",
        "name": "Maximilian Schmidt",
        "affiliation": "University of Stuttgart"
      }
    ]
  },
  "experiments": [
    {
      "id": "experiment0",
      "measurements": [
        {
          "id": "potentiostaticmeasurement0",
          "metadata": [
            {
              "id": "metadata0",
              "parameter": "Galvanostatic Scan",
              "abbreviation": "TITLE",
              "description": "Test Identifier"
            },
            {
              "id": "metadata1",
              "parameter": "7.2.2023",
              "abbreviation": "DATE",
              "description": "Date"

/home/shambles/miniconda3/envs/b07/lib/python3.10/site-packages/sdRDM/base/datamodel.py:213: UserWarning: No 'URL' and 'Commit' specified. This model might not be re-usable.
  warnings.warn(


**Get initial current in mA and initial time in s.**

In [17]:
query_IINIT = '{measurements {metadata (abbreviation: IINIT) {parameter}}}'
I_init= float(b07.query(query_IINIT)['measurements'][0]['metadata'][0]['parameter'])

query_TINIT = '{measurements {metadata (abbreviation: TINIT) {parameter}}}'
T_init = float(b07.query(query_TINIT)['measurements'][0]['metadata'][0]['parameter'])

I_init, T_init

(-200.0, 3600.0)

**Create an instance of the ``MFMParser`` to parse MFM output files and show available files in the selected directory.**

In [18]:
mfmparser = MFMParser(path_to_dataset / 'data')
dict_of_mfm_files = mfmparser.enumerate_available_files()
for index, mfmfile in dict_of_mfm_files.items():
    print(f"{index}: {mfmfile}")

0: Bench-2h-GSS_D9-pH70-1303-C4-2103-1_200_50c_24h
1: PX409-USBH


**Chose specific file and extract the metadata from it using the ``MFMParser``.**

In [19]:
mfm_file = dict_of_mfm_files[0]

In [20]:
model.enums.Unit.MILLILITERPERSECOND.value

'ml/s'

In [21]:
mfm_exp_data_df, mfm_exp_data = mfmparser.extract_exp_data(mfm_file)
# mfm_exp_data_df
# mfm_exp_data_df['datetime']#.dt.to_pydatetime()
# pd.to_datetime(mfm_exp_data_df['datetime'])#.tolist()
# mfm_exp_data_df.dropna().to_csv('mfm_exp_data_df.csv')
# list_of_types = mfmparser.extract_exp_data(mfm_file)


AttributeError: time

In [ ]:
timestamp_object = mfm_exp_data_df.at[100000,'datetime'].to_pydatetime()
timestamp_object
# datetime.fromtimestamp(timestamp_object)

In [ ]:
gcparser = GCParser(path_to_dataset / 'data')
dict_of_gcmd_files = gcparser.enumerate_available_files()
for index, gcmd_file in dict_of_gcmd_files.items():
    print(f"{index}: {gcmd_file}")

In [ ]:
gcmd_file = dict_of_gcmd_files[0]
# print(gcmdparser._available_files[gcm]
# )
metadata, metadata_df = gcparser.extract_metadata(gcmd_file)
metadata_df

In [ ]:
inj_date = metadata_df.at[9, 'column_2']
inj_date

In [ ]:
inj_date_datetime = datetime.strptime(inj_date, '%d-%b-%y, %H:%M:%S')
inj_date_datetime

In [ ]:
vol_flow = []
m = 10
for i, time in enumerate(mfm_exp_data_df['datetime']):
    if time.to_pydatetime() == inj_date_datetime:
        for j in range(i-m,i+m+1):
            vol_flow.append(mfm_exp_data_df.at[j,'flow'])
vol_flow_mean = sum(vol_flow) / (m*2+1)
vol_flow_mean

**Create an instance of the ``GCEDParser`` to parse GC experimental data output files and show available files in the selected directory.**

In [ ]:
gcedparser = GCEDParser(path_to_dataset / 'data')
dict_of_gced_files = gcedparser.enumerate_available_files()
for index, gced_file in dict_of_gced_files.items():
    print(f"{index}: {gced_file}")
# available_files = parser.available_files
# print(available_files)

**Chose specific file and extract the metadata from it using the ``GCParser``.**

In [ ]:
gcedfile = dict_of_gced_files[1]
exp_df = gcedparser.extract_exp_data(gced_file)
exp_df

**Assign peak areas to species.** \
The peak areas recorded by the GC have to be matched with the correct species. The individial ``Area`` is selected by its corresponding ``Peak_Number``. It is possible that the same species is accountable for multiple peaks, i.d. multiple peaks are assigned to the same species.


In [ ]:
assign_peak_dict={
    'CO2': [2],
    'CO': [6],
    'H2': [1],
    'CH4': [3],
    'C2H4': [5],
    # 'C2H6': [4],
}
peak_area_dict = {}
for key, value in assign_peak_dict.items():
    peak_area_dict[key]=exp_df.query('Peak_Number==@value')['Area'].sum()
peak_area_dict

**Set calibration input values and import into the data model.** \
To determine the concentrations of the individual species, a calibration has to be performed in advance to match the individual values for ``Area`` with their corresponding concentrations.

In [ ]:
cali_input_dict={
    'CO2': [[0,38653],[0,50]],
    'CO':[[797,1328,7223],[0.5,1,5]],
    'H2':[[71,153,330], [5,10,20]],
    'CH4':[[5727,11991], [5,10]],
    'C2H4':[[1122,4864,7297], [0.5,2,3]],
    'C2H6':[[0,12168], [0,5]],
}

In [ ]:
cali_dict={}
for key, value in cali_input_dict.items():
    cali_dict[key]=model.Calibration(
        peak_area=[
            model.Data(
                values=value[0],
                # unit=model.
                unit=model.enums.Unit.VOLFRACTION
            )
        ],
        concentration=[
            model.Data(  
                values=value[1],
                unit=model.enums.Unit.VOLFRACTION
            )
        ]    
    )

In [ ]:
b07.general_information.title

In [ ]:
# model.Datamodel.visualize_tree(

In [ ]:
model.Dataset.visualize_tree()

Instantiate the ``Calculator`` module.

In [ ]:
calculator=Calculator(path_to_dataset=path_to_dataset)

Calibrate using the ``calibrate`` method of the ``Calculator`` module.

In [ ]:
cali_result_df=calculator.calibrate(calibration_input_dict=cali_input_dict)
cali_result_df

Calculate volume fractions using the ``calculate_volume_fractions`` method of the ``Calculator`` module.

In [ ]:
vol_frac_df = calculator.calculate_volume_fractions(peak_area_dict=peak_area_dict, calibration_result_df=cali_result_df)
vol_frac_df

Calculate the conversion factors of the individual species using the ``calculate_conversion_factors`` method of the ``Calculator`` module and the correction factors.

In [ ]:
f_corr_dict= {
    'H2':1.01,
    'CO':0.74,
    'CO2':1.00,
    'CH4':0.76,
    # 'C2H4':,
    # 'C2H6':,
}

In [ ]:
vol_frac_df.loc['H2'][0]

In [ ]:
f_conv = calculator.calculate_conversion_factor(volume_fractions=vol_frac_df, correction_factors=f_corr_dict)
f_conv

In [ ]:
vol_flow_r_df = calculator.calculate_vol_flow_r(conversion_factor = f_conv, volumetric_flow_measured = mfm_exp_data_df )
vol_flow_r_df

In [ ]:
# class DirectedGraph:
    
#     def __init__(self, name, connections):
#         self.name = name
#         self.connections = connections
#         self.connec_tuple = []
#         self.make_connectivity()

        
#     def make_connectivity(self):
#         if len(self.connections) == 0: 
#             return([])
#         for connection in self.connections:
#             self.connec_tuple.append((self.name, connection))

In [ ]:
# mod_1 = DirectedGraph(input0, [tubing0])
# mod_2 = DirectedGraph(input1, [tubing1])
# mod_3 = DirectedGraph(tubing0, [valve0])
# mod_4 = DirectedGraph(tubing1, [valve0])
# mod_5 = DirectedGraph(valve0, [tubing2])
# mod_6 = DirectedGraph(fc, [tubing2])
# mod_7 = DirectedGraph(tubing2, [valve1])
# mod_8 = DirectedGraph(valve1, [tubing4])
# mod_9 = DirectedGraph(tubing4, [valve2])
# mod_10 = DirectedGraph(valve2, [tubing5])
# mod_11 = DirectedGraph(tubing5, [gc])
# mod_12 = DirectedGraph(tubing5, [fr])

# modules = [mod_1, mod_2, mod_3, mod_4, mod_5, mod_6, mod_7, mod_8, mod_9, mod_10, mod_11, mod_12]
# module_names = [flow_mod.name for flow_mod in modules]

In [ ]:
# area = exp_df['Area']
# peak_area_dict = {
#     'co2': area.iloc[[1]].sum(),
#     'co': area.iloc[[5,6]].sum(),
#     'h2': area.iloc[[0]].sum(),
#     'ch4': area.iloc[[2,3]].sum(),
#     'c2h4': area.iloc[[4]].sum(),
#     # 'c2h6': area.iloc[[0,1]].sum()
# }
# peak_area_dict

In [ ]:
# flowG = nx.DiGraph()
# flowG.add_nodes_from(module_names)
# for module in modules:
#     flowG.add_edges_from(module.connec_tuple)
# my_pos = nx.spring_layout(flowG, seed = 5) # this fixes the style of the graph, if it's ugly change the seed
# nx.draw(flowG, pos = my_pos, with_labels=True)